In [ ]:
!pip install requests python-dotenv huggingface_hub

In [ ]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add t

In [ ]:
import os
from huggingface_hub import InferenceClient

client = InferenceClient(
    provider="featherless-ai",
)

completion = client.chat.completions.create(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    messages=[
        {
            "role": "user",
            "content": "What is the capital of France?"
        }
    ],
)

print(completion.choices[0].message.content)

 The capital city of France is Paris. Paris is one of the most famous cities in the world and is known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. It is also known for its rich history, art, culture, and fashion. Paris has been the capital city of France since the late 13th century.


# Task
Create a Python script that builds a general health query chatbot using the "mistralai/Mistral-7B-Instruct-v0.2" model. The chatbot should use prompt engineering to act as a helpful medical assistant, provide friendly and clear responses, and include basic safety filters to avoid giving harmful medical advice. The script should allow the user to input queries and receive responses.

## **Define the prompt**




Create a system message string variable with instructions for the LLM to act as a helpful medical assistant and provide general health information while avoiding specific medical advice.



In [ ]:
system_message = """
You are a helpful and friendly general health information assistant. Your purpose is to provide clear and easy-to-understand information about general health topics.

IMPORTANT GUIDELINES:
1. Do NOT provide specific medical diagnoses, treatment plans, or personalized medical advice.
2. Do NOT recommend or endorse specific medications, therapies, or healthcare providers.
3. Always emphasize that the information you provide is for general knowledge only and should not be considered a substitute for professional medical advice.
4. Strongly recommend that the user consult with a qualified healthcare professional for any health concerns, diagnosis, or treatment.
5. Provide responses that are friendly, clear, and based on widely accepted general health knowledge.
6. If a query is asking for specific medical advice or diagnosis, politely reiterate your role as a general information assistant and the importance of consulting a doctor.
"""
print(system_message)


You are a helpful and friendly general health information assistant. Your purpose is to provide clear and easy-to-understand information about general health topics.

IMPORTANT GUIDELINES:
1. Do NOT provide specific medical diagnoses, treatment plans, or personalized medical advice.
2. Do NOT recommend or endorse specific medications, therapies, or healthcare providers.
3. Always emphasize that the information you provide is for general knowledge only and should not be considered a substitute for professional medical advice.
4. Strongly recommend that the user consult with a qualified healthcare professional for any health concerns, diagnosis, or treatment.
5. Provide responses that are friendly, clear, and based on widely accepted general health knowledge.
6. If a query is asking for specific medical advice or diagnosis, politely reiterate your role as a general information assistant and the importance of consulting a doctor.



## **Implement the chatbot logic**

Create a function  that takes a user query, combines it with the system message, and sends it to the LLM using the existing `InferenceClient`.


In [ ]:
def get_chatbot_response(user_query: str, system_message: str) -> str:
    """
    Sends a user query and system message to the LLM and returns the response.

    Args:
        user_query: The user's health query.
        system_message: The system message to guide the LLM's behavior.

    Returns:
        The chatbot's response string.
    """
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_query}
    ]

    completion = client.chat.completions.create(
        model="mistralai/Mistral-7B-Instruct-v0.2",
        messages=messages,
    )

    return completion.choices[0].message.content



In [ ]:
# Example usage (optional, for testing the function)
test_query = "What are the symptoms of the common cold?"
response = get_chatbot_response(test_query, system_message)
print("\nChatbot Response:")
print(response)


Chatbot Response:
 The common cold is a viral infection that affects the nose and throat. Symptoms can vary from person to person, but here are some of the most common signs:

1. Runny or stuffy nose
2. Sore throat
3. Cough
4. Congestion
5. Headache
6. Sneezing
7. Low-grade fever (below 101 degrees Fahrenheit)
8. Body aches
9. Fatigue and weakness

These symptoms typically last for about a week. However, it's important to remember that everyone's experience with the common cold can be different, and some may have more severe symptoms. If you're feeling unwell or have concerns about your health, please consult with a qualified healthcare professional for personalized advice.


## **Add safety filters**

Implement basic checks on the LLM's response to identify and potentially flag or rephrase content that sounds like specific medical advice.


In [ ]:
import re

def filter_response(response: str) -> str:
    """
    Filters the chatbot response for potentially harmful medical advice.

    Args:
        response: The raw chatbot response string.

    Returns:
        The original response or a warning message if potentially harmful content is detected.
    """
    # Basic patterns to look for specific medical advice
    # This is a simplified example and should not be considered a robust medical filter
    medical_advice_patterns = [
        r"take \d+.*(?:milligrams|mg)", # Mentions dosage
        r"use .* (?:for|to treat) .* condition", # Suggests treatment for a condition
        r"apply .* (?:to|on) affected area", # Suggests topical application
        r"consult your doctor about .*", # While often good advice, in combination might suggest specific course
        r"you should .*", # Directive language
        r"it is recommended to .*", # Recommendation language
        r"prescription .*", # Mentions prescriptions
        r"over-the-counter .*", # Mentions over-the-counter drugs
    ]

    warning_message = "Please consult a qualified healthcare professional for medical advice, diagnosis, or treatment. The information provided here is for general knowledge only."

    for pattern in medical_advice_patterns:
        if re.search(pattern, response, re.IGNORECASE):
            return warning_message

    return response



In [ ]:
# Example usage (optional, for testing the function)
test_responses = [
    "You should take 500 milligrams of ibuprofen for your headache.",
    "Apply this ointment to the affected area twice a day.",
    "It is recommended to consult your doctor about this symptom.",
    "The common cold usually resolves within a week.",
    "Getting enough sleep is important for your health.",
]

print("Filtering test responses:")
for res in test_responses:
    filtered_res = filter_response(res)
    print(f"Original: {res}")
    print(f"Filtered: {filtered_res}\n")

Filtering test responses:
Original: You should take 500 milligrams of ibuprofen for your headache.
Filtered: Please consult a qualified healthcare professional for medical advice, diagnosis, or treatment. The information provided here is for general knowledge only.

Original: Apply this ointment to the affected area twice a day.
Filtered: Apply this ointment to the affected area twice a day.

Original: It is recommended to consult your doctor about this symptom.
Filtered: Please consult a qualified healthcare professional for medical advice, diagnosis, or treatment. The information provided here is for general knowledge only.

Original: The common cold usually resolves within a week.
Filtered: The common cold usually resolves within a week.

Original: Getting enough sleep is important for your health.
Filtered: Getting enough sleep is important for your health.



## **User interface**



In [ ]:
while True:
    user_query = input("Enter your health query (or type 'quit' to exit): ")

    if user_query.lower() == 'quit' or user_query.lower() == 'exit':
        break

    raw_response = get_chatbot_response(user_query, system_message)
    filtered_response = filter_response(raw_response)
    print("\nChatbot Response:")
    print(filtered_response)
    print("-" * 30) # Separator for clarity

Enter your health query (or type 'quit' to exit): What causes a sore throat?

Chatbot Response:
 A sore throat can be caused by a variety of factors. Some common causes include:

1. Viruses, such as the flu or common cold
2. Bacteria, such as streptococcus, which can cause strep throat
3. Irritants, such as acid reflux, smoking, or pollution
4. Allergens, such as pollen or pet dander
5. Dry air or excessive talking or singing

Symptoms of a sore throat can include pain or discomfort when swallowing, redness or swelling of the throat, and a scratchy, itchy, or hoarse feeling in the throat. If you're experiencing a sore throat and it's accompanied by other symptoms like fever, body aches, or a cough, it's important to consult with a healthcare professional. They can help determine the underlying cause and provide appropriate treatment. Remember, the information I provide is meant for general knowledge only and should not be used in place of professional medical advice.
------------------

## **Summary:**

*   A system message was successfully created to instruct the LLM to act as a helpful medical assistant, emphasizing the provision of general health information and the importance of consulting healthcare professionals.
*   A Python function `get_chatbot_response` was successfully developed to send user queries and the system message to the specified LLM ("mistralai/Mistral-7B-Instruct-v0.2") and retrieve its response.
*   A basic `filter_response` function was implemented using regular expressions to detect patterns that might indicate specific medical advice and return a warning message if found.
*   A simple user interface loop was created to allow continuous interaction, taking user input, processing it through the chatbot functions, and displaying the filtered response.